# Exercice 2: Classification system with KNN - To Loan or Not To Loan

## Imports

Import some useful libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import math

## a. Getting started

### Data loading

The original dataset comes from the Kaggle's [Loan Prediction](https://www.kaggle.com/ninzaami/loan-predication) problem. The provided dataset has already undergone some processing, such as removing some columns and invalid data. Pandas is used to read the CSV file.

In [2]:
data = pd.read_csv("loandata.csv")

Display the head of the data.

In [3]:
data.head()

,Gender,Married,Education,TotalIncome,LoanAmount,CreditHistory,LoanStatus
0,Male,Yes,Graduate,6091.0,128.0,1.0,N
1,Male,Yes,Graduate,3000.0,66.0,1.0,Y
2,Male,Yes,Not Graduate,4941.0,120.0,1.0,Y
3,Male,No,Graduate,6000.0,141.0,1.0,Y
4,Male,Yes,Graduate,9613.0,267.0,1.0,Y


Data's columns:
* **Gender:** Applicant gender (Male/ Female)
* **Married:** Is the Applicant married? (Y/N)
* **Education:** Applicant Education (Graduate/ Not Graduate)
* **TotalIncome:** Applicant total income (sum of `ApplicantIncome` and `CoapplicantIncome` columns in the original dataset)
* **LoanAmount:** Loan amount in thousands
* **CreditHistory:** Credit history meets guidelines
* **LoanStatus** (Target)**:** Loan approved (Y/N)

### Data preprocessing

Define a list of categorical columns to encode.

In [4]:
categorical_columns = ["Gender", "Married", "Education", "LoanStatus"]

Encode categorical columns using the [`OrdinalEncoder`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html) of scikit learn.

In [5]:
data[categorical_columns] = OrdinalEncoder().fit_transform(data[categorical_columns])

Split into `X` and `y`.

In [6]:
X = data.drop(columns="LoanStatus")
y = data.LoanStatus

Normalize data using the [`StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) of scikit learn.

In [7]:
X[X.columns] = StandardScaler().fit_transform(X[X.columns])

Convert `y` type to `int` 

In [8]:
y = y.astype(int)

Split dataset into train and test sets.

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

## b. Dummy classifier

Build a dummy classifier that takes decisions randomly.

In [10]:
class DummyClassifier():
    
    def __init__(self):
        """
        Initialize the class.
        """
        pass
    
    def fit(self, X, y):
        """
        Fit the dummy classifier.
        
        Parameters
        ----------
        X : Numpy array or Pandas DataFrame of shape (n_samples, n_features)
            Training data.
        y : Numpy array or Pandas DataFrame of shape (n_samples,)
            Target values.
        """
        pass
    
    def predict(self, X):
        """
        Predict the class labels for the provided data.

        Parameters
        ----------
        X : Numpy array or Pandas DataFrame of shape (n_queries, n_features)
            Test samples.

        Returns
        -------
        y : Numpy array or Pandas DataFrame of shape (n_queries,)
            Class labels for each data sample.
        """
        size = np.shape(X)[0]
        return np.random.randint(0, 1, size=size)

Implement a function to evaluate the performance of a classification by computing the accuracy ($N_{correct}/N$).

In [11]:
def accuracy_score(y_true, y_pred):
    ok=0
    y_true = y_true.array
    for i in range(np.shape(y_true)[0]):
        if y_true[i] == y_pred[i]:
            ok+=1
    return ok/np.shape(y_true)[0]

Compute the performance of the dummy classifier using the provided test set.

In [12]:
dc = DummyClassifier()
y_pred = dc.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.3125


## c. K-Nearest Neighbors classifier

Build a K-Nearest Neighbors classifier using an Euclidian distance computation and a simple majority voting criterion.

In [13]:
class KNNClassifier():
    
    def __init__(self, n_neighbors=3):
        """
        Initialize the class.
        
        Parameters
        ----------
        n_neighbors : int, default=3
            Number of neighbors to use by default.
        """
        self.n_neighbors = n_neighbors
    
    def fit(self, X, y):
        """
        Fit the k-nearest neighbors classifier.
        
        Parameters
        ----------
        X : Numpy array or Pandas DataFrame of shape (n_samples, n_features)
            Training data.
        y : Numpy array or Pandas DataFrame of shape (n_samples,)
            Target values.
        """
        self.X=X
        self.y=y
    
    @staticmethod
    def _euclidian_distance(a, b):
        """
        Utility function to compute the euclidian distance.
        
        Parameters
        ----------
        a : Numpy array or Pandas DataFrame
            First operand.
        b : Numpy array or Pandas DataFrame
            Second operand.
        """
        dist=np.linalg.norm(a-b)
        return dist
    
    def predict(self, X):
        """
        Predict the class labels for the provided data.

        Parameters
        ----------
        X : Numpy array or Pandas DataFrame of shape (n_queries, n_features)
            Test samples.

        Returns
        -------
        y : Numpy array or Pandas DataFrame of shape (n_queries,)
            Class labels for each data sample.
        """
        y = {}
        for index, row in X.iterrows():
            #compute distances
            distances = {}
            for i, r in self.X.iterrows() :
                distances[self._euclidian_distance(row, r)] = i
            #sort distances
            distances = dict(sorted(distances.items()))
            #vote
            total = 0
            for i in range(self.n_neighbors-1):
                total+=self.y[list(distances.items())[i][1]]
            total /= self.n_neighbors
            y[index] = round(total)
        return y

Compute the performance of the system as a function of $k = 1...7$.

In [14]:
kc = KNNClassifier()
kc.fit(X_train, y_train)
bestk=0
bestk_acc = 0
for i in range(7):
    kc.n_neighbors=i+1
    y_pred=kc.predict(X_test)
    
    #accuracy computation
    ok=0
    for index, row in y_test.items():
        if row == y_pred.get(index):
            ok+=1
    acc = ok/len(y_pred)
    if(acc>bestk_acc):
        bestk=i+1
        bestk_acc=acc
    print(acc)
    
print(bestk)
kc.n_neighbors=bestk

0.3125
0.3125
0.6354166666666666
0.5520833333333334
0.7395833333333334
0.6979166666666666
0.78125
7


Run the KNN algorithm using only the features `TotalIncome` and `CreditHistory`.

In [15]:
sub_X_train = X_train[["TotalIncome", "CreditHistory"]]
sub_X_test = X_test[["TotalIncome", "CreditHistory"]]

kc.fit(sub_X_train, y_train)
y_pred=kc.predict(sub_X_test)
    
#accuracy computation
ok=0
for index, row in y_test.items():
    if row == y_pred.get(index):
        ok+=1
acc = ok/len(y_pred)
print(acc)

0.75


Re-run the KNN algorithm using the features `TotalIncome`, `CreditHistory` and `Married`.

In [16]:
sub_X_train = X_train[["TotalIncome", "CreditHistory", "Married"]]
sub_X_test = X_test[["TotalIncome", "CreditHistory", "Married"]]

kc.fit(sub_X_train, y_train)
y_pred=kc.predict(sub_X_test)
    
#accuracy computation
ok=0
for index, row in y_test.items():
    if row == y_pred.get(index):
        ok+=1
acc = ok/len(y_pred)
print(acc)

0.75


Re-run the KNN algorithm using all features.

In [17]:
kc.fit(X_train, y_train)
y_pred=kc.predict(X_test)
    
#accuracy computation
ok=0
for index, row in y_test.items():
    if row == y_pred.get(index):
        ok+=1
acc = ok/len(y_pred)
print(acc)

0.78125
